<a href="https://colab.research.google.com/github/omk42/Pattern-Mining/blob/master/clusters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Libraries**

In [0]:
import numpy as np
import matplotlib.pyplot as plt 
from pathlib import Path
from skimage.util.shape import view_as_windows
from skimage.transform import resize
import scipy.io as sio
import pickle

**Configurations**

In [0]:
# conf_rules_path = ROOT + 'output/' + conf_dataset + '/rule.txt'
# conf_invertFile_path = ROOT + 'output/' + conf_dataset + '/invertFile.npy'
# conf_cluster_file = ROOT + 'output/' + conf_dataset + '/clusterFile.pkl'

conf_numTopActivation = 20
conf_supp = 0.1
conf_minLength = 16
conf_maxLength = 18
conf_confid = 30

### FOR LOCAL
ROOT = './MDPM_python/'
conf_output = ROOT + 'shoes_trim_fc7/'
conf_code = '{}m{}n{}q2'.format(str(conf_supp).replace(".", "p"), conf_minLength, conf_maxLength)

conf_rules_path = conf_output + 'rule' + conf_code + '.txt'
conf_invertFile = conf_output + 'invertFile.npy'
conf_cluster_file = conf_output + 'clusters' + conf_code + '.pkl'

In [27]:
conf_rules_path

'./MDPM_python/shoes_trim_fc7/rule0p1m16n18q2.txt'

In [0]:
rules = set()
removed_count = 0

with open (conf_rules_path, 'r') as f: 
  for line in reversed(f.readlines()):
    rule = tuple(int(ele) for ele in line[line.find(' ', line.find(' ')+1)+1:line.find('(')-1].split(' '))    
    # print (rule)
    for i in range(1,len(rule)+1):
      subs = rule[0:i]
      if subs in rules:
        removed_count += 1
        # print ('removed', subs)
        rules.remove(subs)
    rules.add(rule)

In [29]:
len(rules)

14667

In [30]:
removed_count

2294

In [0]:
clusters = list()

with open(conf_invertFile, 'rb') as inv:
  invert = np.load(inv)
  #print (invert)
  for rule in rules:
    index = np.ones ((invert.shape[1],), dtype=bool)
    for feature in rule:
      index = index & np.array(invert[feature], dtype=bool)

    cluster_arr = index.nonzero()[0]
    # if len(cluster_arr) == 1:
    #   print ("hi")
    if len(cluster_arr):
      # print (cluster_arr)
      clusters.append (cluster_arr)


In [32]:
from statistics import mean
print(mean([len(i) for i in clusters]), len(clusters))
# print (len(clusters))

29.553146519397288 14667


In [0]:
with open (conf_cluster_file, 'wb') as f:
  pickle.dump(clusters, f)

For Testing

In [0]:
import numpy as np

invert = np.zeros((10,10))
rows = [0,3,4,4,4,5,2,1,3,9,8,7,7,5,6,0,1,2,6,7,8,9,3,4,7,8,3,4,7,8]
cols = [0,0,2,3,5,7,8,7,6,3,9,9,9,3,1,4,7,2,8,4,5,2,5,5,5,5,9,9,9,9]
for r,c in zip(rows, cols):
  invert[r][c] = 1
with open('/content/sample_invert.npy', 'wb') as f:
  np.save(f, invert)